In [ ]:
import os
import csv

#reads all the episodes in a particular folder and reutns them in a array consisting of arrays with tuples
def read_Episodes(baseFolder):
    episodes = []
    for filename in sorted(os.listdir(baseFolder)):
        filePath = os.path.join(baseFolder, filename)
        episode = []
        with open(filePath, "r", newline="") as file:
            reader = csv.reader(file)
            for row in reader:
                observation = [float(epi) for epi in row]
                episode.append(observation)
            episodes.append(episode)        
    return episodes

In [ ]:
import d3rlpy
import numpy as np
import os

EPIfolder = "../data/episFormula/3. corrected sign"

epis = read_Episodes(EPIfolder)

observations = []
actions = []
rewards = []
terminals = []

for epi in epis:
    for observation in epi:
        observations.append(observation[:-2])
        actions.append(observation[-2:-1])
        rewards.append(observation[-1])
    terminals += [0] * (len(epi) - 1) + [1]

observations=np.array(observations)
actions=np.array(actions)
rewards=np.array(rewards)
terminals=np.array(terminals)

print(observations.shape)

dataset = d3rlpy.dataset.MDPDataset(
    action_space=d3rlpy.constants.ActionSpace.CONTINUOUS,
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)


# create action scaler
action_scaler = d3rlpy.preprocessing.MinMaxActionScaler()
observation_scaler = d3rlpy.preprocessing.MinMaxObservationScaler()

# if you don't use GPU, set device=None instead.
rlModel = d3rlpy.algos.SACConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')

# calculate metrics with training dataset


# combine FileAdapterFactory and TensorboardAdapterFactory
logger_adapter = d3rlpy.logging.CombineAdapterFactory([
   d3rlpy.logging.FileAdapterFactory(root_dir="d3rlpy_logs"),
   d3rlpy.logging.TensorboardAdapterFactory(root_dir="tensorboard_logs"),
])

# use partial episodes as test data
test_episodes = dataset.episodes[:100]

result = rlModel.fit(
    dataset,
    n_steps=100000,
    n_steps_per_epoch=1000,
    evaluators={
        'td_error': d3rlpy.metrics.TDErrorEvaluator(test_episodes),
        'value_scale': d3rlpy.metrics.AverageValueEstimationEvaluator(test_episodes),
        'discounted_advantage': d3rlpy.metrics.DiscountedSumOfAdvantageEvaluator(test_episodes),
        'initial_state': d3rlpy.metrics.InitialStateValueEstimationEvaluator(test_episodes),
        'diff_eval': d3rlpy.metrics.ContinuousActionDiffEvaluator(test_episodes)
    },
    logger_adapter=logger_adapter,
)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(28902, 7)
2025-11-17 18:18.10 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-11-17 18:18.10 [info     ] Action size has been automatically determined. action_size=1
2025-11-17 18:18.10 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=1)
2025-11-17 18:18.10 [debug    ] Fitting observation scaler...  observation_scaler=min_max
2025-11-17 18:18.10 [debug    ] Fitting action scaler...       action_scaler=min_max
2025-11-17 18:18.10 [debug    ] Building models...            
2025-11-17 18:18.14

Epoch 1/500: 100%|██████████| 1000/1000 [00:26<00:00, 38.12it/s, critic_loss=0.937, actor_loss=-3.27, temp=0.871, temp_loss=1.41]


2025-11-17 18:18.42 [info     ] SAC_20251117181814: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.004574637174606324, 'time_algorithm_update': 0.02105555605888367, 'critic_loss': 0.9413413234651089, 'actor_loss': -3.284698159635067, 'temp': 0.8704106951355934, 'temp_loss': 1.4057204422950744, 'time_step': 0.025853415489196778, 'td_error': 1.3224049726089755, 'value_scale': 4.973161024103092, 'discounted_advantage': -6.3298876679793326, 'initial_state': 5.375366687774658, 'diff_eval': 13339.042409626574} step=1000
2025-11-17 18:18.42 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251117181814\model_1000.d3


Epoch 2/500: 100%|██████████| 1000/1000 [00:24<00:00, 41.34it/s, critic_loss=1.76, actor_loss=-5.92, temp=0.672, temp_loss=1.01]


...

Epoch 232/500: 100%|██████████| 1000/1000 [00:24<00:00, 41.66it/s, critic_loss=3.29e+18, actor_loss=-4.94e+9, temp=6.25e+7, temp_loss=-2.46e+5]


2025-11-17 20:00.48 [info     ] SAC_20251117181814: epoch=232 step=232000 epoch=232 metrics={'time_sample_batch': 0.004384138584136963, 'time_algorithm_update': 0.019001256227493286, 'critic_loss': 3.284793637254067e+18, 'actor_loss': -4945169210.368, 'temp': 62454696.264, 'temp_loss': -236234.6015, 'time_step': 0.023631055116653442, 'td_error': 1.2209942192882227e+18, 'value_scale': 4755445221.988658, 'discounted_advantage': -7350646678.970222, 'initial_state': 6241115136.0, 'diff_eval': 77468.3660647531} step=232000
2025-11-17 20:00.49 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251117181814\model_232000.d3


Epoch 233/500: 100%|██████████| 1000/1000 [00:25<00:00, 39.07it/s, critic_loss=3.42e+18, actor_loss=-5.03e+9, temp=6.36e+7, temp_loss=-3.27e+5]


2025-11-17 20:01.16 [info     ] SAC_20251117181814: epoch=233 step=233000 epoch=233 metrics={'time_sample_batch': 0.004525190353393555, 'time_algorithm_update': 0.020398086547851562, 'critic_loss': 3.416867934405927e+18, 'actor_loss': -5030851700.736, 'temp': 63573582.42, 'temp_loss': -315250.18240625, 'time_step': 0.025168591022491454, 'td_error': 1.2648439747885775e+18, 'value_scale': 4837842062.275992, 'discounted_advantage': -7491860087.827849, 'initial_state': 6349764096.0, 'diff_eval': 77502.66695555307} step=233000
2025-11-17 20:01.16 [info     ] Model parameters are saved to d3rlpy_logs\SAC_20251117181814\model_233000.d3


Epoch 234/500:  35%|███▍      | 349/1000 [00:08<00:15, 41.61it/s, critic_loss=3.57e+18, actor_loss=-5.09e+9, temp=6.44e+7, temp_loss=-3.49e+5]